In [76]:
import re
import warnings
warnings.filterwarnings('ignore')

import os
import json
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
data_dir = "../data/new"
train_data = pd.read_csv(f"{data_dir}/new_train_ver3.csv")
test_data = pd.read_csv(f"{data_dir}/new_test_ver3.csv")

In [3]:
train_data['is_test'] = 0
test_data['is_test'] = 1

data = pd.concat([train_data, test_data])
data['is_test'].value_counts() 

0    1118822
1       9272
Name: is_test, dtype: int64

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 9271
Data columns (total 43 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   본번                      1128094 non-null  float64
 1   부번                      1128094 non-null  float64
 2   아파트명                    1128094 non-null  object 
 3   전용면적                    1128094 non-null  float64
 4   층                       1128094 non-null  int64  
 5   건축년도                    1128094 non-null  int64  
 6   도로명                     1128094 non-null  object 
 7   해제사유발생일                 6195 non-null     float64
 8   등기신청일자                  1128094 non-null  object 
 9   거래유형                    1128094 non-null  object 
 10  부동산유형                   1128094 non-null  object 
 11  분양형태                    1128094 non-null  object 
 12  k-관리방식                  1128094 non-null  object 
 13  k-복도유형                  1128094 non-null  object 
 14  k-난방방

In [5]:
# '주소1' 컬럼 생성
data['주소1'] = data.apply(lambda row: f"{row['구']} {row['동']} {int(row['본번'])}-{int(row['부번'])}" if row['부번'] != 0 else f"{row['구']} {row['동']} {int(row['본번'])}", axis=1)

# '주소2' 컬럼 생성
data['주소2'] = data.apply(lambda row: f"{row['구']} {row['도로명']}", axis=1)

# 1.부동산 이름 찾기

In [6]:
# '주소2'의 고유값 추출
unique_address = list(data['주소1'].unique())
print(len(unique_address))

# '주소2'의 고유값들을 기준으로 '아파트명'이 'unknown'인 행들을 선별
# unknown_apartment_rows = data[(data['주소2'].isin(unique_address)) & (data['아파트명'] == 'unknown')]
# print(len(unknown_apartment_rows))

8956


In [7]:
url = "https://land.seoul.go.kr/land/wskras/generalInfo.do"
browser = webdriver.Chrome()
browser.get(url)

# 자치구 선택 요소 찾기
select_element_gu = browser.find_element(By.ID, 'selSgg')
select_gu = Select(select_element_gu)

# 자치구의 이름과 value를 저장할 딕셔너리
gu_dict = {}
for option in select_gu.options:
    gu_name = option.text
    if gu_name != "자치구 선택":
        value = option.get_attribute('value')
        gu_dict[gu_name] = value

# 자치구 별로 동의 옵션 값 가져오기
gd_dict = {}
for gu_name, gu_value in gu_dict.items():
    # 자치구 선택
    select_gu.select_by_value(gu_value)
    time.sleep(1)  # 페이지 업데이트를 기다림
    
    # 동 선택 요소 찾기
    select_element_dong = browser.find_element(By.ID, 'selBjdong')
    select_dong = Select(select_element_dong)
    
    dong_dict = {}
    for option in select_dong.options:
        dong_name = option.text
        if dong_name != "동 선택":
            value = option.get_attribute('value')
            dong_dict[dong_name] = value
    gd_dict[gu_name] = dong_dict

browser.quit()

In [8]:
with open(f'{data_dir}/seoul_gu_dong.json', 'w', encoding='utf-8') as f:
    json.dump(gd_dict, f, ensure_ascii=False, indent=4)

In [9]:
with open(f'{data_dir}/seoul_gu_dong.json', 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

In [82]:
url = "https://land.seoul.go.kr/land/wskras/generalInfo.do"
browser = webdriver.Chrome()
browser.get(url)

for idx, address in enumerate(unique_address):
    parts = address.split()
    gu_name = parts[0].strip()
    dong_name = parts[1].strip()
    bonbun_bubun = parts[2].strip()
    print(f"{idx:>08}")
    print(f"[원본] : 서울특별시 {gu_name} {dong_name} {bonbun_bubun}")

    if "-" in bonbun_bubun:
        buns = bonbun_bubun.split('-')
        bonbun = buns[0]
        bubeon = buns[1]
    else:
        bonbun = bonbun_bubun
        bubeon = None

    if gu_name in loaded_data:
        dong_dict = loaded_data[gu_name]
        if dong_name in dong_dict:
            gu_value = gu_dict[gu_name]
            dong_value = dong_dict[dong_name]

            ## 구, 동, 본번-부번
            select_element_gu = browser.find_element(By.ID, 'selSgg')
            select_gu = Select(select_element_gu)
            select_gu.select_by_value(gu_value)
            time.sleep(1)

            select_element_dong = browser.find_element(By.ID, 'selBjdong')
            select_dong = Select(select_element_dong)
            select_dong.select_by_value(dong_value)

            bonbeon_element = browser.find_element(By.ID, "bonbeon")
            bonbeon_element.clear()
            bonbeon_element.send_keys(bonbun)

            if not bubeon is None:
                bubeon_element = browser.find_element(By.ID, "bubeon")
                bubeon_element.clear()
                bubeon_element.send_keys(bubeon)
            else:
                bubeon_element = browser.find_element(By.ID, "bubeon")
                bubeon_element.clear()

            btn = browser.find_element(By.ID, "btnSearch")
            btn.click()
            time.sleep(1)

            tab2 = browser.find_element(By.ID, "tab2")
            tab2.click()
            time.sleep(2)

            apart_name = browser.find_element(By.ID, "tdBldNm").text.strip()
            print(f"건물명 : {apart_name}")
            if len(apart_name) == 0:
                print()
                continue

            if re.search('[a-zA-Z]', apart_name):
                sub_url = "https://www.naver.com/"
                sub_browser = webdriver.Chrome()
                sub_browser.get(sub_url)
                time.sleep(2)

                sub_query = f"서울특별시 {gu_name} {dong_name} {bonbun_bubun}"
                search_box = sub_browser.find_element(By.CLASS_NAME, "search_input_box")
                search_element = search_box.find_element(By.TAG_NAME, "input")
                search_element.send_keys(sub_query)
                search_element.send_keys(Keys.RETURN)
                time.sleep(2)

                end_box = sub_browser.find_element(By.CLASS_NAME, "end_box")
                apart_name = end_box.find_element(By.CLASS_NAME, "build_name").text

            sub_url = "https://land.naver.com/"
            sub_browser = webdriver.Chrome()
            sub_browser.get(sub_url)
            time.sleep(2)

            sub_query = f"{gu_name} {dong_name} {apart_name}"
            print(sub_query)
            search_element = sub_browser.find_element(By.ID, "queryInputHeader")
            search_element.send_keys(sub_query)
            search_element.send_keys(Keys.RETURN)
            time.sleep(2)

            summary_info = sub_browser.find_elements(By.ID, 'summaryInfo')
            if not summary_info:
                item_list = sub_browser.find_element(By.CLASS_NAME, "item_list--search")
                items = item_list.find_elements(By.CLASS_NAME, "item")[1:]
                items[0].click()
                time.sleep(2)

            btn_div = sub_browser.find_element(By.CLASS_NAME, 'complex_detail_link')
            btns = btn_div.find_elements(By.TAG_NAME, 'button')
            btn = btns[0].click()
            time.sleep(2)

            ## 아파트명
            title = sub_browser.find_element(By.ID, "complexTitle")
            apart_name = title.text
            data.loc[data['주소1'] == address, '아파트명'] = apart_name
            print(f' -아파트명 : {apart_name}')

            table = sub_browser.find_element(By.CLASS_NAME, 'info_table_wrap')
            trs = table.find_elements(By.TAG_NAME, 'tr')
            
            ## 세대수
            tr0_data = trs[0].find_element(By.TAG_NAME, 'td').text
            tr0_splits = tr0_data[:-1].split('(')

            rooms = tr0_splits[0][:-2]
            dongs = tr0_splits[1][1]
            data.loc[data['주소1'] == address, 'k-전체동수'] = dongs
            data.loc[data['주소1'] == address, 'k-전체세대수'] = rooms
            print(f' -k-전체동수 : {dongs}')
            print(f' -k-전체세대수 : {rooms}')

            ## 건설사
            tr3_data = trs[3].find_element(By.TAG_NAME, 'td').text
            data.loc[data['주소1'] == address, 'k-건설사'] = tr3_data
            print(f' -k-건설사 : {tr3_data}')

            tr4_data = trs[4].find_element(By.TAG_NAME, 'td').text
            heat_type = tr4_data.split(',')[0]
            data.loc[data['주소1'] == address, 'k-난방방식'] = heat_type
            print(f' -k-난방방식 : {heat_type}')

            # break
            print()
            sub_browser.quit()
browser.quit()

00000000
[원본] : 서울특별시 강남구 개포동 658-1
건물명 : 개포6차우성아파트
강남구 개포동 개포6차우성아파트
 -아파트명 : 우성6차
 -k-전체동수 : 8
 -k-전체세대수 : 270
 -k-건설사 : 우성건설(주)
 -k-난방방식 : 개별난방

00000001
[원본] : 서울특별시 강남구 개포동 652
건물명 : 우성아파트
강남구 개포동 우성아파트
 -아파트명 : 우성3차
 -k-전체동수 : 5
 -k-전체세대수 : 405
 -k-건설사 : 우성건설(주)
 -k-난방방식 : 중앙난방

00000002
[원본] : 서울특별시 강남구 개포동 12-2
건물명 : 개포자이
강남구 개포동 개포자이
 -아파트명 : 개포자이
 -k-전체동수 : 4
 -k-전체세대수 : 212
 -k-건설사 : LG건설(주)
 -k-난방방식 : 지역난방

00000003
[원본] : 서울특별시 강남구 개포동 141
건물명 : 

00000004
[원본] : 서울특별시 강남구 개포동 187
건물명 : 개포주공아파트
강남구 개포동 개포주공아파트
 -아파트명 : 개포주공5단지
 -k-전체동수 : 6
 -k-전체세대수 : 940
 -k-건설사 : 대한주택공사
 -k-난방방식 : 지역난방

00000005
[원본] : 서울특별시 강남구 개포동 185
건물명 : 개포주공아파트
강남구 개포동 개포주공아파트
 -아파트명 : 개포주공5단지
 -k-전체동수 : 6
 -k-전체세대수 : 940
 -k-건설사 : 대한주택공사
 -k-난방방식 : 지역난방

00000006
[원본] : 서울특별시 강남구 논현동 245
건물명 : 논현동동양파라곤
강남구 논현동 논현동동양파라곤
 -아파트명 : 논현동양파라곤
 -k-전체동수 : 4
 -k-전체세대수 : 203
 -k-건설사 : 동양고속건설(주)
 -k-난방방식 : 개별난방

00000007
[원본] : 서울특별시 강남구 논현동 58-2
건물명 : 마일스 디오빌
강남구 논현동 마일스 디오빌
 -아파트명 : 마일스디오빌(주상복합)
 -k-전체동수 : 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".end_box"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010be750e8 chromedriver + 5169384
1   chromedriver                        0x000000010be6cfba chromedriver + 5136314
2   chromedriver                        0x000000010b9e936c chromedriver + 402284
3   chromedriver                        0x000000010ba36740 chromedriver + 718656
4   chromedriver                        0x000000010ba36a01 chromedriver + 719361
5   chromedriver                        0x000000010ba7bbc4 chromedriver + 1002436
6   chromedriver                        0x000000010ba59add chromedriver + 862941
7   chromedriver                        0x000000010ba78f57 chromedriver + 991063
8   chromedriver           